# Pathway Diagram Builder

This can help building a flux diagram of pathway more efficiently with the aid of Chemkin data. It could be seen as a complementary code for the mechanism analyzer, offering the result:

- The flux to important products from certain species

- The ratio of the flux between the reactant flux and a certain flux

In [1]:
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.rmg.pdep import PDepReaction
from rmgpy.chemkin import loadChemkinFile, getSpeciesIdentifier
from rmgpy.tools.extractInfoFromckcsv import getConcentrationDictFromCKCSV, getROPFromCKCSV, getFluxGraphEdgesDict
from rmgpy.species import Species
from IPython.display import display, Image
import os
import numpy as np
from scipy.integrate import simps
import networkx as nx
import re
import pydot

# Needed for Cantera simulation
from rmgpy.tools.canteraModel import Cantera
from rmgpy.quantity import Quantity
import cantera as ct
from scipy import sparse

## MECHANISM INPUT (Run Once per Mechanism Analyzed)

Supported Methodologies:

    - 'chemkin'
    - 'cantera'


Necessary Files (all methods): 

    - chemkin file (chem.inp)
    - species_dictionary.txt
    
Files Necessary for Chemkin Method:

    - chemkin simulation file (*.ckcsv)


Place these files under ./data/pathway_analysis/`mech` and input `mech` variable in the next cell.

In [4]:
########## User Input ##########
methodology = 'chemkin'
mech = 'CPO_8.1'
################################

In [6]:
mechPath = os.path.join('./', 'data', 'pathway_analysis', mech)
chemkinPath= os.path.join(mechPath, 'chem.inp')
dictionaryPath = os.path.join(mechPath, 'species_dictionary.txt')

# extract the data to a RMG model
model = CoreEdgeReactionModel()
model.core.species, model.core.reactions = loadChemkinFile(chemkinPath, dictionaryPath, checkDuplicates=False)

# generate pairs for reactions that don't have flux pairs
for rxn in model.core.reactions:
    if not rxn.pairs: rxn.generatePairs()
        
if methodology == 'chemkin':
    # Abstract information from chemkin output
    ckcsvPath= mechPath + '/CKSoln.ckcsv'
    firstColDict, spc_mf_dict = getConcentrationDictFromCKCSV(ckcsvPath)
    first_col_dict, spc_total_rop_dict, spc_rop_dict = getROPFromCKCSV(ckcsvPath)
        
elif methodology == 'cantera':
    rmg_cantera_model = Cantera(speciesList=model.core.species, reactionList=model.core.reactions)
    rmg_cantera_model.loadModel()

# Create species_identifier_dict for the core species    
species_identifier_dict = {}
for i, s in enumerate(model.core.species):
    species_identifier_dict.update({getSpeciesIdentifier(model.core.species[i]): s})
    
# Create reaction_identifier_dict
rxn_identifier_dict = {}
for i, s in enumerate(model.core.reactions):
    rxn_identifier_dict.update({s.index:i})

## END MECHANISM INPUT
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Helper Tool (Optional)
### Species search

Run the following code to help determine the necessary species identifiers for the INPUT section

In [7]:
############## User Input (optional) ##############
spc_index = 4
###################################################

spc = model.core.species[spc_index]
print 'The species identifier at index {0} is: {1}'.format(spc_index, getSpeciesIdentifier(spc))
display(spc)

The species identifier at index 4 is: H(2)


In [ ]:
############## User Input (optional) ##############
species_smiles =  "C1C[C](O)CC1"
###################################################

if species_smiles:
    search_species = Species().fromSMILES(species_smiles)
    for key, key_item in species_identifier_dict.iteritems():
        if key_item.isIsomorphic(search_species):
            display(search_species)
            print 'has label as {0}'.format(key)
            species_smiles = None
            break
if species_smiles:
    print "Related species is not involved in the model."

In [ ]:
############## User Input (optional) ##############
species_label = 'C5H8O(684)'
###################################################

if species_label:
    try:
        display(species_identifier_dict[species_label])
        print "has label as {0}".format(species_label)
        
    except KeyError:
        print "The label is invalid."

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## CANTERA INPUT (Run Once per Cantera Simulation / Time Point)

- `reactor_type` the zero-dimensional Cantera reaction type with the following available options:
            * 'IdealGasReactor' (adiabatic, isochoric reactor)
            * 'IdealGasConstPressureReactor' (adiabatic, isobaric reactor)
            * 'IdealGasConstPressureTemperatureReactor' (isothermal, isobaric reactor)


- `reactor_time` the time point you want to investigate

- `mol_frac_list` a dictionary with species identifiers as keys and initial mole fractions as values

- `reactor_temp` the initial reactor temperature as an RMG Quantity object e.g. Quantity(value, unit_string)

- `reactor_pressure` the initial reactor pressure as an RMG Quantity object e.g. Quantity(value, unit_string)

- `integrated_flux` True if users want to use time-integrated fluxes (integrated up to the investigated time point) in kmol/kg_total for analysis. False if users want to use instantaneous fluxes in kmol/m3-s at the investigated time point for analysis

In [ ]:
########################### User Input ###########################
reactor_type = 'IdealGasConstPressureTemperatureReactor'
reactor_time = Quantity(6*3600, 's')

mol_frac_list = {'PDD(1)':0.25,
                 'Toluene(2)':0.25,
                 'C11(36)':0.5,
                }

reactor_temp = Quantity(673, 'K')
reactor_pressure = Quantity(300, 'bar')
integrated_flux = True
##################################################################

In [ ]:
solution = rmg_cantera_model.model # Cantera framework for thermo and kinetics data

# Set the reactor initial conditions
solution.TPX = reactor_temp.value_si, reactor_pressure.value_si, mol_frac_list
sparse_coefs = sparse.csr_matrix(solution.product_stoich_coeffs()) - sparse.csr_matrix(solution.reactant_stoich_coeffs()) # Stoichiometry matrix for ROP calcs

# Choose reactor
if reactor_type == 'IdealGasReactor':
    cantera_reactor = ct.IdealGasReactor(solution)
elif reactor_type == 'IdealGasConstPressureReactor':
    cantera_reactor = ct.IdealGasConstPressureReactor(contents=solution)
elif reactor_type == 'IdealGasConstPressureTemperatureReactor':
    cantera_reactor = ct.IdealGasConstPressureReactor(contents=solution, energy='off')
else:
    raise Exception('Other types of reactor conditions are currently not supported')
    
reactor_net = ct.ReactorNet([cantera_reactor])


# Run the Cantera simulation and calculate the ROPs
if integrated_flux: # sparse_rop is in kmol/kg_total for each reaction
    iteration = 0
    rop_iteration_modulus = 1 # Change if you want to "integrate" the rops less often (speed vs. accuracy trade-off)
    sparse_net_rates = sparse.diags(solution.net_rates_of_progress, format='csr')
    sparse_production_old = sparse_coefs.dot(sparse_net_rates)
    sparse_rops = sparse.csr_matrix(sparse_production_old.shape)
    volume = solution.volume_mass
    time = reactor_net.time
    
    while reactor_net.time < reactor_time.value_si:
        reactor_net.step()
        iteration += 1
        if iteration % rop_iteration_modulus == 0:
            sparse_net_rates = sparse.diags(solution.net_rates_of_progress, format='csr')
            sparse_production_new = sparse_coefs.dot(sparse_net_rates)
            sparse_rops += 0.5 * (sparse_production_new * solution.volume_mass + sparse_production_old * volume) * (reactor_net.time - time)
            volume = solution.volume_mass
            time = reactor_net.time
            sparse_production_old = sparse_production_new
            
        
else: # sparse_rop is in kmol/m3-s
    reactor_net.advance(reactor_time.value_si)
    sparse_net_rates = sparse.diags(solution.net_rates_of_progress, format='csr')
    sparse_rops = sparse_coefs.dot(sparse_net_rates)
    

## END CANTERA INPUT
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## CHEMKIN INPUT (Run Once per Chemkin Time Point)



- `time_investigated` the time point or time scale (for integrated flux) you want to investigate

- `reactant_name` the chemkin name of your initial species/reactant

- `T` the reaction temperature in K

- `P` the reaction pressure in Pa if the mechanism includes pressure-dependent reactions

- `showIntegratedFlux` True if users want to use time-integrated fluxes (integrated up to the investigated time point) in mole for analysis. False if users want to use instantaneous fluxes in mole/cm3/s at the investigated time point for analysis

In [47]:
a = sparse.diags([1,2,3,4])
a
a.data[0,0]

1.0

In [8]:
time_investigated = 0.004 # unit: second
reactant_name = 'CPO(1)'
T = 1173 # unit: K
P = 8.97*101325 # unit: Pa
showIntegratedFlux = True

In [41]:
timepoint_index = (np.abs(firstColDict['Time_(sec)']-time_investigated)).argmin()
timeList = firstColDict['Time_(sec)']
Plist = firstColDict['Pressure_(atm)']
Tlist = firstColDict['Temperature_(k)']
Vlist = firstColDict['Volume_(cm3)']

print "Investigated time point is {0} secs".format(timeList[timepoint_index])
print "At this moment, the reactant's mole fraction is {}.".format(spc_mf_dict[reactant_name][timepoint_index])
    
# Create network
G = nx.DiGraph()

# Create production and consumption flux dictionary of each species
species_total_produce_flux_dict = {}
species_total_consume_flux_dict = {}

for species_symbol, species in species_identifier_dict.iteritems():

    # Initialize the dictionary
    species_total_produce_flux_dict.update({species_symbol: 0})
    species_total_consume_flux_dict.update({species_symbol: 0})
    
    # Don't do anything for nonreactive species
    if species_symbol in ['He', 'Ne', 'Ar']:
        continue
    
    if methodology == 'chemkin':
        rxn_rops_of_species = spc_rop_dict[species_symbol]
    elif methodology == 'cantera':
        rxn_rops_of_species = sparse_rops[species.index,:]
    
    # Iterate all reaction in which the species involved
    for rxn_rop in rxn_rops_of_species:
        
        if methodology == 'chemkin':
            # Search for the reaction index from the table head 
            rxn_index_search = re.search('#[0-9]+_',rxn_rop[0])
            rxn_index = int(rxn_index_search.group()[1:-1])
        if methodology == 'cantera':
            rxn_index = rxn_rop
        
        # Find the exact reaction in the core
        reaction = model.core.reactions[rxn_identifier_dict[rxn_index]]
        
        # Integrated flux or instananeous flux will be obtained for this reaction
        if showIntegratedFlux:
            flux = simps(rxn_rop[1][:timepoint_index+1],timeList[:timepoint_index+1],)
        else:
            flux = rxn_rop[1][timepoint_index]
        
        # Positive flux means produce species while negative flux means consume species
        if flux >= 0:
            species_total_produce_flux_dict[species_symbol] += flux
        else:
            species_total_consume_flux_dict[species_symbol] += -flux
            
            # Digraph is established here to avoid double counting issue
            if species in reaction.reactants:
                for product in reaction.products:
                    node = getSpeciesIdentifier(product)
                    if G.has_edge(species_symbol, node):
                        
                        # Each edge of G has two properties, total flux and reaction list
                        G[species_symbol][node]['total_flux'] += -flux
                        G[species_symbol][node]['rxn'].append(reaction)
                    else:
                        G.add_edge(species_symbol, node, total_flux = -flux, rxn = [reaction,])
            else:
                for reactant in reaction.reactants:
                    node = getSpeciesIdentifier(reactant)
                    if G.has_edge(species_symbol, node):
                        G[species_symbol][node]['total_flux'] += -flux
                        G[species_symbol][node]['rxn'].append(reaction)
                    else:
                        G.add_edge(species_symbol, node, total_flux = -flux, rxn = [reaction,])

# Calculate the consumption of the indicated reactant
reactantConsumption = species_total_consume_flux_dict[reactant_name]

Investigated time point is 0.00399 secs
At this moment, the reactant's mole fraction is 0.004734242.
C5H6(3446) 3446
C6H8(1889) 1889
S(1009) 1009
C4H6(1048) 1048
CH3(13) 13
C5H7(3408) 3408
C6H7(3048) 3048
C4H6(1023) 1023
C6H8(1968) 1968
C6H9(1887) 1887
C5H4(3495) 3495
C5H9(428) 428
C4H5O(906) 906
C4H8(706) 706
C5H8O(690) 690
C6H8(2791) 2791
HCCO(54) 54
C6H6(3107) 3107
C5H9O(730) 730
NC3H7(92) 92
C5H8O(683) 683
C2H5(36) 36
C4H6(208) 208
C6H9(3300) 3300
CPO-2(677) 677
C6H7(2000) 2000
C6H7(2204) 2204
C6H7(2432) 2432
C5H5(2613) 2613
CH3CO(46) 46
C6H9(2832) 2832
S(1001) 1001
C4H7(194) 194
S(1780) 1780
C6H8(1583) 1583
C4H6(1065) 1065
C6H8(1963) 1963
C6H7(2222) 2222
S(1350) 1350
C5H6(1745) 1745
C6H7(2785) 2785
C5H7(331) 331
S(1390) 1390
S(1347) 1347
C5H6(2184) 2184
C4H5(1598) 1598
C5H6O(324) 324
C6H8(2147) 2147
C3H4(1631) 1631
S(1919) 1919
S(1866) 1866
C6H9(1886) 1886
C5H8O(685) 685
C4H5(2423) 2423
C6H8(1894) 1894
C5H9(400) 400
C3H5(101) 101
S(1284) 1284
S(1259) 1259
C4H6(1123) 1123
C6H8(1878

## Generate Flux diagram
### INPUT

- `spc_num_first_layer` the number of species, which have the largest fluxes from/to the species you want to look at of the first layer

- `spc_num_other_layer` same idea as `spc_num_first_layer`, but for other layers

- `pct_coverage` the percentage of the flux be covered from/to the species. 

- `pct_initial` the threshold to determine where the branches end

- `hierarchy` the number of layer to be built

- `preference` indicates the species you want to display in the diagram, 'carbon only' will exclude species with out carbon atom

- `png_name` is the .png file name you want to save as

- `verbose` set to be True if you want to get a symbolic pathway diagram as well

In [ ]:
spc_num_first_layer = 9
spc_num_other_layer = 4
pct_coverage = 0.99
pct_initial = 0.01
hierarchy = 5
preference = 'carbon only'
png_name = 'flux.png'
verbose = False

Display the flux diagram in labels

In [ ]:
from_species = True
species = reactant_name
pct_coverage_default = pct_coverage

# Structure is a tree like list with each item containing the 
# species in the corresponding layer
structure = [[species,],]

# spc_involved will record what species have already showed up
# in this network
spc_involved = []

# DG is a smaller Digraph used to store the diagram information only
DG = nx.DiGraph()

# Generate the network layer by layer
level = 0
while level <= hierarchy:
    for current_node in structure[level]:
        
        # If the species already be included, no need to look at its fluxes again
        if current_node in spc_involved:
            continue
        else:
            spc_involved.append(current_node)
            
            # if the network has already been extend to the desired level, no
            # need to look at as well
            if level == hierarchy:
                continue
        
        # It is very important to extend the network as much as we can for
        # the first layer, otherwise we may lose track for important path
        if level ==0:
            pct_coverage = 0.99
            spc_num = spc_num_first_layer
        else:
            pct_coverage = pct_coverage_default
            spc_num =spc_num_other_layer
        
        if verbose:
            print "Level {0}".format(level + 1)   
            print '************************************************************************'
        
        structure.append([])
        nextNode_flux_list = [(next_node, G[current_node][next_node]['total_flux']) for next_node in G[current_node]]
        sorted_flux_list = sorted(nextNode_flux_list, key=lambda tup: -tup[1])
        coverage = 0
        
        # The coverage calculation considering the repeated flux
        # e.g. A -> B + C, the flux_sum will include flux(A -> B)
        # as well as flux(A -> C)
        flux_sum_with_repeat = 0
        for item in sorted_flux_list:
            flux_sum_with_repeat += item[1]
        
        # No out flux from this species 
        if species_total_consume_flux_dict[current_node] == 0:
            if verbose:
                print "{0} is the end node with no flux.".format(current_node)
            continue
        
        # Iterate each flux from large to small until reach the threshold of 
        # species number or 
        for index in range(spc_num):
            structure[level+1].append(sorted_flux_list[index][0])
            step_flux = sorted_flux_list[index][1]
            ratio_initial_species = step_flux / reactantConsumption
            coverage += step_flux / flux_sum_with_repeat
            
            DG.add_edge(current_node, sorted_flux_list[index][0], flux_pct = ratio_initial_species)
            if verbose:
                print "{0} --> {1} \t\t {2:.3f}%".\
            format(current_node, sorted_flux_list[index][0], ratio_initial_species*100)
            
            # The out flux of the species is too small, makes the outfluxes of these species
            # neglectable
            if index == 0 and ratio_initial_species <= pct_initial:
                if verbose:
                    print "{0} is the branch end with little flux.".format(current_node)
                break

            # If coverage threshold is satisfied, we will look at next species of this layer
            if coverage >= pct_coverage:
                break
        if verbose:
            print '************************************************************************'
        
    level +=1

Generate the flux diagram and save it as `flux.png` with pydot

In [ ]:
# Basic Setting for this diagram
graph = pydot.Dot('flux_diagram', graph_type='digraph', overlap="false")
graph.set_rankdir('LR')
graph.set_fontname('sans')
graph.set_fontsize('10')

# Create a path to save the image of each species
speciesPath = mechPath +'/Species'
if not os.path.exists(speciesPath):
    os.mkdir(mechPath +'/Species')

# Create a forbidden list that species would not be shown in the diagram
forbidden = []
if preference == 'carbon only':
    for index in spc_involved:
        species = species_identifier_dict[index]
        if species.molecule[0].getNumAtoms(element='C') == 0:
            forbidden.append(index)
if preference == 'nitrogen only':
    for index in spc_involved:
        species = species_identifier_dict[index]
        if species.molecule[0].getNumAtoms(element='N') == 0:
            forbidden.append(index)
    
# Add a node for each species
for index in spc_involved:
    species = species_identifier_dict[index]
    if index in forbidden:
        continue
    imagePath = os.path.join(os.path.abspath(mechPath), 'Species', '{0}.png'.format(index))
    with open(imagePath, 'w') as f:
        f.write(species._repr_png_())
    if os.path.exists(imagePath):
        node = pydot.Node(name=index, shape='rectangle', fontcolor='transparent',\
                          image=imagePath)
    else:
        node = pydot.Node(name=index, shape='rectangle')
    graph.add_node(node)
       
# Add an edge for each species-species rate
for reactantIndex, productIndex in DG.edges:
    if (reactantIndex in forbidden) or (productIndex in forbidden):
        continue
    reactant = species_identifier_dict[reactantIndex]
    product = species_identifier_dict[productIndex]
    edge = pydot.Edge(reactantIndex, productIndex)
    edge_flux = DG[reactantIndex][productIndex]['flux_pct'] * 100
    edge.set_label("{0:.2f}%".format(edge_flux))
    graph.add_edge(edge)


# Generate the coordinates for all of the nodes using the specified program
program = 'dot'     
graph = pydot.graph_from_dot_data(graph.create_dot(prog=program))[0]

graph.write_png(os.path.join(mechPath, png_name))

display the flux diagram
- hint: double click the image to zoom in/out

In [ ]:
Image(filename=os.path.join(mechPath, png_name))

### INPUT 

- `from_species` the direction "from a species" (from_species = True) or "to a species" (from_species = False)

- `species` the chemkin name of the species

- `spc_num` the number of species, which have the largest fluxes from/to the species you want to look at

- `pct_coverage` the percentage of the flux be covered from/to the species. 

- `example_rxn_num` the number of example reactions to be shown

The number of output fluxes will be decided on both last criteria and only the smaller one will be reported

In [ ]:
from_species = True
species = "C4H7(191)"
spc_num = 6
pct_coverage = 0.99
example_rxn_num = 2

In [ ]:
if from_species:
    direction = "From"
    total_species_flux = species_total_consume_flux_dict[species]
else:
    direction = "To"
    total_species_flux = species_total_produce_flux_dict[species]
        
if showIntegratedFlux:
    unit = "mole"   
else:
    unit = "mole/cm3/s"

print "At this moment, the reactant's total molar consumption is {0:.3E} {1}.".\
    format(reactantConsumption, unit)
print ''

if showIntegratedFlux:
    print '{0} {1} given integrated flux'.format(direction, species)
else:
    print '{0} {1} given instantaneous flux'.format(direction , species)
print '======================================'
print 'You are trying to find the top {0} fluxes.'.format(spc_num)
print ''

if from_species:
    nextNode_flux_list = [(next_node, G[species][next_node]['total_flux']) for next_node in G[species]]
    sorted_flux_list = sorted(nextNode_flux_list, key=lambda tup: -tup[1])
else:
    prev_nodes = []
    for node1 in G:
        if species in G[node1]:
            prev_nodes.append(node1)
    prevNode_flux_list = [(prev_node, G[prev_node][species]['total_flux']) for prev_node in prev_nodes]
    sorted_flux_list = sorted(prevNode_flux_list, key=lambda tup: -tup[1])

print "The total flux {0} {1} is {2: .4E} {3}".format(direction, species, total_species_flux, unit)

coverage = 0
repeated_flux = 0
for item in sorted_flux_list:
    repeated_flux += item[1]

for index in range(spc_num):
    step_flux = sorted_flux_list[index][1]
    ratio_initial_species = step_flux / reactantConsumption
    coverage += step_flux / repeated_flux
    
    if from_species:
        species_pair = (species, sorted_flux_list[index][0])
    else:
        species_pair = (sorted_flux_list[index][0], species)
    
    print '************************************************************************'
    print "No.{0}".format(index + 1)
    print "{0} --> {1} \t\t {2:.3E} {3} \t {4:.3f}%".\
    format(species_pair[0], species_pair[1], step_flux, unit, ratio_initial_species*100)
    
    rxn_count = 0
    reaction_list = G[species_pair[0]][species_pair[1]]['rxn']
    for rxn in reaction_list:
        print ''
        print "Example reaction: rxn#{0}".format(rxn.index)
        print rxn
        display(rxn)
        
        if rxn.kinetics.isPressureDependent:
            print 'Forward rate coefficient at {0} K: {1:.2E} mole/cm3/s'.\
            format(T, rxn.getRateCoefficient(T, P))
            reverseRate = rxn.generateReverseRateCoefficient()
            print 'Reverse rate coefficient at {0} K: {1:.2E} mole/cm3/s'.\
            format(T, reverseRate.getRateCoefficient(T, P))
        else:
            print 'Forward rate coefficient at {0} K: {1:.2E} mole/cm3/s'.\
            format(T, rxn.getRateCoefficient(T))
            reverseRate = rxn.generateReverseRateCoefficient()
            print 'Reverse rate coefficient at {0} K: {1:.2E} mole/cm3/s'.\
            format(T, reverseRate.getRateCoefficient(T))
            
        rxn_count += 1
        if rxn_count >= example_rxn_num:
            break
            
    if coverage >= pct_coverage:
        print ""
        print "{0} major fluxes are found satisfying your coverage.".format(index+1)
        break